# General tool for creating Muscle Analysis setup xml file

> Alex Woodall<br>
> Auckland Bioengineering Institute<br>
> Auckland, New Zealand

This notebook will create a muscle analysis setup xml file using the OpenSim API in Python.

_Note:_ This is written in Python 3.7.4 with OpenSim 4.0. Make sure you have read the [Python 3 OpenSim API Setup](OpenSim4.0Python3API.ipynb) and testing that the OpenSim API is working with Python 3.

__Assuming that the motion capture data was pre-processed in Nexus to remove gaps and duplicate markers.__

This notebook is better used as a function call where the input variables have been created prior to calling.

Start by importing the necessary libraries

In [1]:
import numpy as np
import opensim as osim
from xml.dom import minidom
import sys

sys.path.insert(1, r'Functions')  # add to path
from xml_shorten import xml_shorten

## Requirements

To generate the muscle analysis setup xml file, we will need five items:
- __trial__: the trial name, e.g., "12_Mar_ss_12ms_01"
- __model__: the model name, e.g., "AB08"
- __directory__: the output directory name
- __time_range__: the start and end times of the trial
- __cut_off_freq__: the low pass cut-off frequency (Hz)

In [2]:
# Establish input variables example
trial = '_12Mar_ss_12ms_01'
model = 'AB08'
directory = 'C:\\Users\\alexw\\Dropbox\\ABI\\Level_8_Lab\\OpenSim Tools\\ProcessingTrialDataFromVicon\\Output'
time_range = [15.3, 30.455]
cut_off_freq = 10

## Establish output, time, and filtering variables

In [6]:
# Get analyze tool
analyze_tool = osim.AnalyzeTool()

# Set tool name
new_analyze_tool_name = model + trial
analyze_tool.setName(new_analyze_tool_name)

# Set the opensim model name
analyze_tool.setModelFilename(directory + "\\" + model + "\\" + model + ".osim")

# Set the results directory
analyze_tool.setResultsDir(directory + "\\" + model + "\\" + trial)

# Set the external loads file
external_loads_file = directory + "\\" + model + "\\" + trial + "\\" + trial + 'ExternalLoads.xml'
analyze_tool.setExternalLoadsFileName(external_loads_file)

# Set the coordinates file
coord_file = directory + "\\" + model + "\\" + trial + "\\" + trial + 'IKResults.mot'
analyze_tool.setCoordinatesFileName(coord_file)

# Set low pass cut-off frequency, NOTE: Must be a double (np.float64)
analyze_tool.setLowpassCutoffFrequency(np.float64(cut_off_freq))

# Set the time range, NOTE: Must be a double (np.float64)
analyze_tool.setInitialTime(np.float64(time_range[0]))
analyze_tool.setFinalTime(np.float64(time_range[-1]))

## Add muscle analysis

In [7]:
analysis_set = analyze_tool.getAnalysisSet()

muscle_analysis = osim.MuscleAnalysis()

muscle_analysis.setStartTime(round(np.float64(time_range[0]), 3))
muscle_analysis.setEndTime(np.float64(time_range[-1]))
muscle_analysis.setComputeMoments(True)

analysis_set.cloneAndAppend(muscle_analysis);

## Write to an XML setup file

In [8]:
xml_setup_path = directory + "\\" + model + "\\" + trial + "\\" + trial + "MuscleAnalysisSetup.xml"
analyze_tool.printToXML(xml_setup_path)

''' Temporary fix to set compute moments to true and to remove numerical inaccuracy in times '''

dom = minidom.parse(xml_setup_path)
analysis_set = dom.getElementsByTagName("AnalysisSet")
analysis_set_child = analysis_set.item(0)

objects_set = analysis_set_child.getElementsByTagName("objects")
objects_set_child = objects_set.item(0)

muscle_analysis = objects_set_child.getElementsByTagName("MuscleAnalysis")
muscle_analysis_child = muscle_analysis.item(0)

muscle_analysis_child.getElementsByTagName("compute_moments")[0].firstChild.nodeValue = "true"

dom.getElementsByTagName("initial_time")[0].firstChild.nodeValue = time_range[0]
dom.getElementsByTagName("final_time")[0].firstChild.nodeValue = time_range[-1]
muscle_analysis_child.getElementsByTagName("start_time")[0].firstChild.nodeValue = time_range[0]
muscle_analysis_child.getElementsByTagName("end_time")[0].firstChild.nodeValue = time_range[-1]

with open(xml_setup_path, 'w') as xml_file:
    dom.writexml(xml_file, addindent='\t', newl='\n', encoding='UTF-8')
    
# Using minidom will often create unneeded white space
xml_shorten(xml_setup_path)